In [1]:
import numpy as np
import rasterio
from pysheds.grid import Grid
import logging
from scipy import ndimage

In [2]:


# 设置日志
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def simplified_hand(dem, rivers):
    """
    简化版的 HAND (Height Above Nearest Drainage) 计算
    """
    # 创建一个掩码，其中河流像素为 True，其他为 False
    mask = rivers.astype(bool)

    # 计算每个非河流像素到最近河流像素的距离
    dist, _ = ndimage.distance_transform_edt(~mask, return_distances=True, return_indices=True)

    # 创建一个与 DEM 相同大小的数组来存储 HAND 值
    hand = np.zeros_like(dem)

    # 对于每个非河流像素，计算其高度与最近河流像素高度的差
    for i in range(dem.shape[0]):
        for j in range(dem.shape[1]):
            if not mask[i, j]:
                # 找到最近的河流像素
                y, x = np.unravel_index(np.argmin(dist[i, j]), dem.shape)
                # 计算高度差
                hand[i, j] = dem[i, j] - dem[y, x]

    return hand

def identify_rivers_and_alluvial(dem_path, flow_accumulation_threshold, slope_threshold, hand_threshold, alluvial_slope_threshold):
    """
    使用 PySheds 识别河流和冲积区
    """
    logging.info("Starting river and alluvial area identification")
    
    # 初始化 Grid 对象
    grid = Grid.from_raster(dem_path)
    
    # 读取 DEM 数据
    dem = grid.read_raster(dem_path)
    
    # 填充 DEM 中的坑和平坦区域
    pit_filled_dem = grid.fill_pits(dem)
    flooded_dem = grid.fill_depressions(pit_filled_dem)
    inflated_dem = grid.resolve_flats(flooded_dem)
    
    # 计算流向 (使用 D8 方法)
    flow_direction = grid.flowdir(inflated_dem)
    
    # 计算流量累积
    flow_accumulation = grid.accumulation(flow_direction)
    
    # 计算坡度
    cell_slopes = grid.cell_slopes(inflated_dem, flow_direction)
    
    # 识别河流
    rivers = (flow_accumulation > flow_accumulation_threshold) & (cell_slopes < slope_threshold)
    
    # 计算简化版的 HAND
    hand = simplified_hand(inflated_dem, rivers)
    
    # 识别冲积区
    alluvial_areas = (hand < hand_threshold) & (cell_slopes < alluvial_slope_threshold) & (~rivers)
    
    logging.info("River and alluvial area identification completed")
    return rivers, alluvial_areas, dem

def main(dem_path, output_path, flow_accumulation_threshold=50000, slope_threshold=0.05, 
         hand_threshold=5, alluvial_slope_threshold=0.02):
    """
    主函数
    """
    logging.info(f"Processing DEM: {dem_path}")
    logging.info(f"Output will be saved to: {output_path}")

    # 识别河流和冲积区
    rivers, alluvial_areas, dem = identify_rivers_and_alluvial(
        dem_path, flow_accumulation_threshold, slope_threshold, hand_threshold, alluvial_slope_threshold
    )

    # 使用 rasterio 获取坐标信息
    with rasterio.open(dem_path) as src:
        meta = src.meta

    # 创建结果栅格
    result = np.zeros_like(dem, dtype=np.int32)
    result[rivers] = 1  # 河流区域
    result[alluvial_areas] = 2  # 冲积区

    # 创建新的tif文件保存结果
    meta.update({
        'dtype': 'int32',
        'nodata': 0,
        'compress': 'lzw',
        'count': 1
    })

    logging.info(f"Saving results to {output_path}")
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(result, 1)

    logging.info("Processing completed successfully")


In [3]:
if __name__ == '__main__':
    dem_path = r"C:\Users\Runker\Desktop\River\dem.tif"
    output_path = r"C:\Users\Runker\Desktop\River\river_alluvial_area1.tif"
    main(dem_path, output_path)

2024-08-22 10:07:21,583 - INFO - Processing DEM: C:\Users\Runker\Desktop\River\dem.tif
2024-08-22 10:07:21,584 - INFO - Output will be saved to: C:\Users\Runker\Desktop\River\river_alluvial_area1.tif
2024-08-22 10:07:21,584 - INFO - Starting river and alluvial area identification
2024-08-22 10:24:17,061 - INFO - River and alluvial area identification completed
2024-08-22 10:24:17,468 - INFO - Saving results to C:\Users\Runker\Desktop\River\river_alluvial_area1.tif
2024-08-22 10:24:19,688 - INFO - Processing completed successfully
